# Literary Application

In [1]:
%load_ext literary.notebook

In [1]:
from jupyter_core.application import JupyterApp

In [3]:
class LiteraryApp(JupyterApp):
    name = "literary"
    description = "A Literary application"